In [1]:
# Importamos los paquetes necesarios para la practica
import cv2  
import math 
from ultralytics import YOLO

In [ ]:
# Cargamos el modelo ya entrenado para la deteccion de vehiculos y personas
model = YOLO("yolo11m.pt")

In [ ]:
# Probamos el modelo con un video en el que aparecen coches y personas
filename = "prueba.mp4"
results = model(filename, show=True)

cv2.destroyAllWindows()
cv2.waitKey(0)

In [2]:
# Ahora importamos un modelo sin entrenar para poder entrenarlo y detectar matriculas
non_trained_model = YOLO("yolo11n.yaml")

# Antes de comenzar el entreno comprobamos que la GPU esta disponible
import torch
print(torch.cuda.is_available())

True


In [ ]:
# Entrenamos el modelo para la detección de matriculas

results = non_trained_model.train(
    data='LICENSE_PLATE_DETECTION.yaml',
    epochs=60,                          # Número de épocas
    imgsz=640,                           # Tamaño de la imagen de entrada
    batch=16,                            # Tamaño del lote
    patience=10,                        # Paciencia para la detención temprana
    device=0                            # Usar GPU 0 para el entrenamiento
)

In [3]:
# Si ya lo hemos entrenado previamente, podemos cargar los pesos guardados
trained_model = YOLO("C:/Users/edani/OneDrive/Escritorio/Visión por Computador/Practica_4/runs/detect/train5/weights/best.pt")

In [ ]:
# Comprobamos que el modelo detecta las matriculas en una imagen de prueba
IMAGE_TO_PREDICT = 'https://img.motor.mapfre.es/wp-content/uploads/2018/05/matricula_en_mal_estado_y_riesgo_de_multa.jpg'

print(f"Iniciando predicción en: {IMAGE_TO_PREDICT}")

# Ejecutar la predicción y guardar la visualización
results = trained_model.predict(
    source=IMAGE_TO_PREDICT,
    save=True
)

In [ ]:
results = trained_model("C:/Users/edani/OneDrive/Escritorio/Visión por Computador/Practica_4/LicenseDetection.mp4", show=True)

cv2.destroyAllWindows()
cv2.waitKey(0)

In [ ]:

# Ahora combinamos ambos modelos para detectar coches y matriculas en un video
video_path = 'C:/Users/edani/OneDrive/Escritorio/Visión por Computador/Practica_4/LicenseDetection.mp4'
cap = cv2.VideoCapture(video_path)

# Colores para las detecciones
COLOR_COCHE = (255, 0, 0) # Azul
COLOR_MATRICULA = (0, 255, 0) # Verde

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detección de Coches
    results_coches = model(frame, verbose=False)
    
    # Detección de Matrículas
    results_matriculas = trained_model(frame, verbose=False)

    # Ahora combinamos las detecciones y dibujamos los cuadros en el frame
    
    # Dibujar coches
    for r in results_coches:
        for box in r.boxes:
            # Obtiene las coordenadas y las convierte a enteros
            x1, y1, x2, y2 = [int(i) for i in box.xyxy[0]]
            
            clase_nombre = model.names[int(box.cls)]
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR_COCHE, 1)
            cv2.putText(frame, clase_nombre, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_COCHE, 1)

    # Dibujar matrículas
    for r in results_matriculas:
        for box in r.boxes:
            # Obtiene las coordenadas y las convierte a enteros
            x1, y1, x2, y2 = [int(i) for i in box.xyxy[0]]
            
            # Asume que la clase 'license_plate' (matrícula) es la relevante
            clase_nombre = trained_model.names[int(box.cls)] 
            
            cv2.rectangle(frame, (x1, y1), (x2, y2), COLOR_MATRICULA, 1)
            cv2.putText(frame, clase_nombre, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, COLOR_MATRICULA, 1)


    # Mostrar el frame procesado
    cv2.imshow('Deteccion de Coches y Matriculas', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()